In [1]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import mediapipe as mp

# --- Khởi tạo MediaPipe Pose ---
mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(static_image_mode=False,
                             model_complexity=0,
                             smooth_landmarks=True,
                             enable_segmentation=False,
                             min_detection_confidence=0.35,
                             min_tracking_confidence=0.35)

# --- Các hàm phụ trợ ---
def sort_corners(pts_array):
    if not isinstance(pts_array, np.ndarray): pts_array = np.array(pts_array)
    if pts_array.ndim == 3: pts_array = pts_array.reshape(-1, 2)
    if pts_array.shape[0] != 4: return None
    rect = np.zeros((4, 2), dtype="float32")
    s = pts_array.sum(axis=1)
    rect[0], rect[2] = pts_array[np.argmin(s)], pts_array[np.argmax(s)]
    idx_tl, idx_br = np.argmin(s), np.argmax(s)
    remaining_indices = [i for i in range(4) if i not in [idx_tl, idx_br]]
    if len(remaining_indices) == 2:
        pt1, pt2 = pts_array[remaining_indices[0]], pts_array[remaining_indices[1]]
        if pt1[1] < pt2[1]: rect[1], rect[3] = pt1, pt2
        elif pt2[1] < pt1[1]: rect[1], rect[3] = pt2, pt1
        else: rect[1], rect[3] = (pt1, pt2) if pt1[0] > pt2[0] else (pt2, pt1)
    else: return None
    return rect.astype(np.int32)

def is_point_in_box(px, py, x1, y1, x2, y2, tol=0):
    return (x1 - tol <= px <= x2 + tol and y1 - tol <= py <= y2 + tol)

def is_box_intersecting_polygon(box, polygon_pts):
    x1, y1, x2, y2 = box
    box_poly_pts = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]], dtype=np.float32)
    if polygon_pts.ndim == 2: poly_cv = polygon_pts.reshape(-1,1,2).astype(np.float32)
    elif polygon_pts.ndim == 3 and polygon_pts.shape[1]==1 and polygon_pts.shape[2]==2: poly_cv = polygon_pts.astype(np.float32)
    else: return False
    for pt in box_poly_pts:
        if cv2.pointPolygonTest(poly_cv, (float(pt[0]), float(pt[1])), False) >= 0: return True
    for pt in poly_cv.reshape(-1,2):
        if x1 <= pt[0] <= x2 and y1 <= pt[1] <= y2: return True
    return False

def draw_badminton_court_lines(image, W, H, color=(255, 255, 255), thickness=1):
    if W <= 0 or H <= 0: return
    cv2.rectangle(image, (0, 0), (W - 1, H - 1), color, thickness)
    cv2.line(image, (0, H // 2), (W - 1, H // 2), color, thickness) # Net line

    # Standard badminton court ratios (length 13.4m, width 6.1m for doubles)
    court_length_ratio_ref = 13.4
    court_width_ratio_ref = 6.1

    single_line_offset_x = int((0.46 / court_width_ratio_ref) * W)
    cv2.line(image, (single_line_offset_x, 0), (single_line_offset_x, H - 1), color, thickness)
    cv2.line(image, (W - 1 - single_line_offset_x, 0), (W - 1 - single_line_offset_x, H - 1), color, thickness)

    short_service_line_offset_y = int((1.98 / court_length_ratio_ref) * H)
    y_ssl_top = H // 2 - short_service_line_offset_y
    y_ssl_bottom = H // 2 + short_service_line_offset_y
    cv2.line(image, (single_line_offset_x, y_ssl_top), (W - 1 - single_line_offset_x, y_ssl_top), color, thickness)
    cv2.line(image, (single_line_offset_x, y_ssl_bottom), (W - 1 - single_line_offset_x, y_ssl_bottom), color, thickness)

    cv2.line(image, (W // 2, y_ssl_top), (W // 2, 0), color, thickness) # Center line top half
    cv2.line(image, (W // 2, y_ssl_bottom), (W // 2, H - 1), color, thickness) # Center line bottom half

    long_service_line_doubles_offset_y = int((0.76 / court_length_ratio_ref) * H)
    y_lsld_top = long_service_line_doubles_offset_y
    y_lsld_bottom = H - 1 - long_service_line_doubles_offset_y
    cv2.line(image, (0, y_lsld_top), (W - 1, y_lsld_top), color, thickness)
    cv2.line(image, (0, y_lsld_bottom), (W - 1, y_lsld_bottom), color, thickness)

def process_video_for_court_boundary(video_path):
    model = YOLO("yolov8n.pt")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Lỗi: Không thể mở video từ '{video_path}'"); return

    # --- Heatmap Grid Configuration ---
    GRID_COLS = 30
    GRID_ROWS = 40

    # Preserve original dest_rect dimensions as per user's last code
    dest_rect_width = 610
    dest_rect_height = int(dest_rect_width * (67.1 / 61.0))
    dest_rect_corners_dst = np.array([
        [[0,0]],[[dest_rect_width-1,0]],
        [[dest_rect_width-1,dest_rect_height-1]],[[0,dest_rect_height-1]]], dtype=np.float32)

    clahe_heatmap = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_video = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    last_known_roi_corners = None

    default_warped_display = np.zeros((dest_rect_height, dest_rect_width, 3), dtype=np.uint8)
    
    # Preserve original default_schematic dimensions as per user's last code
    default_schematic_display_width = 300
    default_schematic_display_height = int(300 * (13.4/6.1)) # This uses badminton aspect ratio
    default_schematic_court_img = np.zeros((default_schematic_display_height, default_schematic_display_width, 3), dtype=np.uint8)
    # Draw faint court lines on default schematic as a placeholder
    draw_badminton_court_lines(default_schematic_court_img, default_schematic_display_width, default_schematic_display_height, (70,70,70), 1)
    cv2.putText(default_schematic_court_img, "No schematic data", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (220,220,220), 1)

    all_schematic_positions_history = []
    frame_count = 0
    start_time_overall = time.time()
    user_exited_during_video = False # Flag for user pressing 'q' during video

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Hết video.")
            break # Exit main loop

        display_frame = np.copy(frame)
        warped_display = default_warped_display.copy()
        # schematic_court_img will be determined based on H_matrix presence

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        v_enh = clahe_video.apply(v)
        hsv_enh = cv2.merge([h, s, v_enh])
        mask = cv2.inRange(hsv_enh, np.array([30,30,30]), np.array([90,255,255]))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8), iterations=2)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((11,11),np.uint8), iterations=3)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        current_roi_corners = None
        if contours:
            cnt = max(contours, key=cv2.contourArea)
            # Area threshold to filter small contours
            if cv2.contourArea(cnt) > 0.005 * frame.shape[0] * frame.shape[1]:
                poly = cv2.approxPolyDP(cnt, 0.025 * cv2.arcLength(cnt, True), True)
                if len(poly) == 4: current_roi_corners = sort_corners(poly)

        final_roi_corners = current_roi_corners if current_roi_corners is not None else last_known_roi_corners
        if final_roi_corners is not None: last_known_roi_corners = final_roi_corners

        players_on_court, player_midpoints_original_with_color = [], []
        player_transformed_points_with_color = []

        H_matrix = None
        # Default schematic dimensions if H_matrix is not found or invalid
        schematic_w_actual = default_schematic_display_width
        schematic_h_actual = default_schematic_display_height
        limit_x1_w, limit_y1_w = 0, 0
        limit_x2_w, limit_y2_w = schematic_w_actual, schematic_h_actual # Default to full default schematic size

        if final_roi_corners is not None and final_roi_corners.shape == (4,2):
            yolo_res = model(frame, classes=[0], verbose=False)
            persons = []
            for res in yolo_res:
                for box_obj in res.boxes:
                    b = box_obj.xyxy[0].cpu().numpy().astype(int)
                    if is_box_intersecting_polygon(b, final_roi_corners):
                        persons.append({'box':b, 'area':(b[2]-b[0])*(b[3]-b[1])})
            players_on_court = sorted(persons, key=lambda x: x['area'], reverse=True)[:2]
            colors = [(255,0,0), (0,0,255)] # Player 1 Blue, Player 2 Red

            for i, p_data in enumerate(players_on_court):
                x1_p,y1_p,x2_p,y2_p = p_data['box'] # Renamed to avoid conflict
                cv2.rectangle(display_frame, (x1_p,y1_p), (x2_p,y2_p), colors[i], 2)
                roi_img = frame[y1_p:y2_p, x1_p:x2_p]
                if roi_img.size == 0: continue
                pose_res = pose_detector.process(cv2.cvtColor(roi_img, cv2.COLOR_BGR2RGB))
                if pose_res.pose_landmarks:
                    lm = pose_res.pose_landmarks.landmark
                    h_r, w_r, _ = roi_img.shape
                    try:
                        lh, rh = lm[mp_pose.PoseLandmark.LEFT_HEEL.value], lm[mp_pose.PoseLandmark.RIGHT_HEEL.value]
                        if lh.visibility > 0.5 and rh.visibility > 0.5:
                            lx_f, ly_f = int(lh.x*w_r)+x1_p, int(lh.y*h_r)+y1_p
                            rx_f, ry_f = int(rh.x*w_r)+x1_p, int(rh.y*h_r)+y1_p
                            tol = int(0.05*min(x2_p-x1_p,y2_p-y1_p))
                            if is_point_in_box(lx_f,ly_f,x1_p,y1_p,x2_p,y2_p,tol) and \
                               is_point_in_box(rx_f,ry_f,x1_p,y1_p,x2_p,y2_p,tol):
                                mid_x, mid_y = int((lx_f+rx_f)/2), int((ly_f+ry_f)/2)
                                player_midpoints_original_with_color.append(((mid_x,mid_y), colors[i]))
                                cv2.circle(display_frame, (mid_x,mid_y), 7, colors[i], -1)
                                cv2.circle(display_frame, (mid_x,mid_y), 8, (255,255,255),1)
                    except (IndexError, Exception): pass
            
            try:
                H_matrix_cand, _ = cv2.findHomography(final_roi_corners.astype(np.float32), dest_rect_corners_dst)
                if H_matrix_cand is not None and H_matrix_cand.shape == (3,3):
                    H_matrix = H_matrix_cand
                    warped_display = cv2.warpPerspective(frame, H_matrix, (dest_rect_width, dest_rect_height))
                    
                    h_w_display, w_w_display = dest_rect_height, dest_rect_width
                    off_x_r, off_y_r = 0.081, 0.06 # Original offsets
                    limit_x1_w_calc = int(w_w_display*off_x_r)
                    limit_y1_w_calc = int(h_w_display*off_y_r)
                    limit_x2_w_calc = w_w_display-int(w_w_display*off_x_r)
                    limit_y2_w_calc = h_w_display-int(h_w_display*off_y_r)

                    # Update actual limits and schematic dimensions only if H_matrix is valid
                    limit_x1_w, limit_y1_w = limit_x1_w_calc, limit_y1_w_calc
                    limit_x2_w, limit_y2_w = limit_x2_w_calc, limit_y2_w_calc
                    schematic_w_actual = limit_x2_w - limit_x1_w
                    schematic_h_actual = limit_y2_w - limit_y1_w
                    
                    lim_cnr_w = np.array([[[limit_x1_w,limit_y1_w]],[[limit_x2_w,limit_y1_w]],
                                          [[limit_x2_w,limit_y2_w]],[[limit_x1_w,limit_y2_w]]], dtype=np.float32)
                    
                    if not np.array_equal(H_matrix, np.zeros((3,3))) and not np.array_equal(H_matrix, np.eye(3)):
                        try:
                            H_inv = np.linalg.inv(H_matrix)
                            lim_cnr_orig = cv2.perspectiveTransform(lim_cnr_w, H_inv)
                            if lim_cnr_orig is not None:
                                cv2.polylines(display_frame, [np.round(lim_cnr_orig).astype(np.int32)], True, (0,255,255),3)
                        except np.linalg.LinAlgError: H_matrix = None 
                    else: H_matrix = None 

                    cv2.rectangle(warped_display, (limit_x1_w,limit_y1_w), (limit_x2_w,limit_y2_w), (0,255,0),2)

                    if player_midpoints_original_with_color and H_matrix is not None:
                        pts_orig = np.array([[list(d[0]) for d in player_midpoints_original_with_color]], dtype=np.float32)
                        pts_trans = cv2.perspectiveTransform(pts_orig, H_matrix)
                        if pts_trans is not None:
                            for i_pt, (orig_data, trans_pt) in enumerate(zip(player_midpoints_original_with_color, pts_trans[0])):
                                color_loop = orig_data[1]
                                tx, ty = int(trans_pt[0]), int(trans_pt[1])
                                
                                if 0 <= tx < w_w_display and 0 <= ty < h_w_display:
                                    cv2.circle(warped_display, (tx,ty),7,color_loop,-1)
                                    cv2.circle(warped_display,(tx,ty),8,(255,255,255),1)
                                
                                if limit_x1_w <= tx < limit_x2_w and limit_y1_w <= ty < limit_y2_w:
                                    player_transformed_points_with_color.append(((tx,ty), color_loop))
                                    schematic_px = tx - limit_x1_w
                                    schematic_py = ty - limit_y1_w
                                    if schematic_w_actual > 0 and schematic_h_actual > 0 and \
                                       0 <= schematic_px < schematic_w_actual and \
                                       0 <= schematic_py < schematic_h_actual:
                                        all_schematic_positions_history.append((schematic_px, schematic_py))
            except Exception: H_matrix = None

        # Determine base for schematic court image
        if H_matrix is not None and schematic_w_actual > 0 and schematic_h_actual > 0:
            current_heatmap_base = np.zeros((schematic_h_actual, schematic_w_actual, 3), dtype=np.uint8)
        else:
            current_heatmap_base = default_schematic_court_img.copy()


        # Heatmap generation grid-based
        if H_matrix is not None and schematic_w_actual > 0 and schematic_h_actual > 0:
            if all_schematic_positions_history:
                heatmap_grid_counts = np.zeros((GRID_ROWS, GRID_COLS), dtype=np.float32)
                
                cell_width_on_schematic = schematic_w_actual / GRID_COLS
                cell_height_on_schematic = schematic_h_actual / GRID_ROWS

                if cell_width_on_schematic > 0 and cell_height_on_schematic > 0:
                    for hx, hy in all_schematic_positions_history:
                        grid_col = int(hx / cell_width_on_schematic)
                        grid_row = int(hy / cell_height_on_schematic)
                        
                        grid_col = min(max(grid_col, 0), GRID_COLS - 1)
                        grid_row = min(max(grid_row, 0), GRID_ROWS - 1)
                        heatmap_grid_counts[grid_row, grid_col] += 1
                
                max_grid_count = np.max(heatmap_grid_counts)
                if max_grid_count > 0:
                    normalized_grid_counts = heatmap_grid_counts / max_grid_count
                    
                    upscaled_intensity_map_float = cv2.resize(normalized_grid_counts, 
                                                              (schematic_w_actual, schematic_h_actual), 
                                                              interpolation=cv2.INTER_LINEAR)
                    
                    upscaled_intensity_map_uint8 = (upscaled_intensity_map_float * 255).astype(np.uint8)
                    heatmap_enhanced_uint8 = clahe_heatmap.apply(upscaled_intensity_map_uint8)
                    heatmap_blurred_uint8 = cv2.GaussianBlur(heatmap_enhanced_uint8, (15, 15), 0) # Original blur kernel
                    
                    current_heatmap_base = cv2.applyColorMap(heatmap_blurred_uint8, cv2.COLORMAP_JET)
            
            # Draw court lines on top of the heatmap or black base
            draw_badminton_court_lines(current_heatmap_base, schematic_w_actual, schematic_h_actual, color=(255,255,255), thickness=1)
            
            # Draw current player positions on top
            for transformed_pt_data in player_transformed_points_with_color:
                (tx, ty), player_color = transformed_pt_data 
                pt_x_on_schematic = tx - limit_x1_w
                pt_y_on_schematic = ty - limit_y1_w
                if 0 <= pt_x_on_schematic < schematic_w_actual and 0 <= pt_y_on_schematic < schematic_h_actual:
                    cv2.circle(current_heatmap_base, (pt_x_on_schematic, pt_y_on_schematic), 7, player_color, -1)
                    cv2.circle(current_heatmap_base, (pt_x_on_schematic, pt_y_on_schematic), 8, (0,0,0), 1)
        
        schematic_court_img = current_heatmap_base
        
        if H_matrix is None and final_roi_corners is not None:
            cv2.polylines(display_frame, [final_roi_corners.reshape(-1,1,2)], True, (0,0,255),1)

        cv2.imshow('Phat hien san va VDV', display_frame)
        cv2.imshow('San Dau Da Lam Phang (Top-down)', warped_display)
        cv2.imshow('San Cau Long Mo Phong (Heatmap)', schematic_court_img)
        
        frame_count += 1
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            user_exited_during_video = True
            print("Đã nhấn 'q' trong quá trình xử lý video. Thoát.")
            break

    # After the main loop (video ended or 'q' was pressed)
    if not user_exited_during_video and frame_count > 0:
        print("Video đã kết thúc. Nhấn 'q' trên một trong các cửa sổ để đóng.")
        while True:
            key_after_video = cv2.waitKey(30) & 0xFF
            if key_after_video == ord('q'):
                break
            # Check if windows were closed manually, if so, break
            # This can be problematic and OS-dependent, so a simpler 'q' to quit is often more robust.
            try:
                if cv2.getWindowProperty('Phat hien san va VDV', cv2.WND_PROP_VISIBLE) < 1 or \
                   cv2.getWindowProperty('San Dau Da Lam Phang (Top-down)', cv2.WND_PROP_VISIBLE) < 1 or \
                   cv2.getWindowProperty('San Cau Long Mo Phong (Heatmap)', cv2.WND_PROP_VISIBLE) < 1:
                    print("Một cửa sổ đã bị đóng thủ công.")
                    break
            except cv2.error: # Catch error if a window was already destroyed
                print("Lỗi truy cập thuộc tính cửa sổ, có thể cửa sổ đã bị đóng.")
                break
            time.sleep(0.01) # Reduce CPU usage in this waiting loop

    cap.release()
    pose_detector.close()
    cv2.destroyAllWindows()
    total_time = time.time() - start_time_overall
    if total_time > 0 and frame_count > 0:
        print(f"Xử lý hoàn tất. Tổng frames: {frame_count}, Tổng thời gian: {total_time:.2f}s, FPS trung bình: {frame_count/total_time:.2f}")
    else: print("Xử lý hoàn tất.")

if __name__ == '__main__':
    video_file_path = 'demo.mp4' 
    try:
        with open(video_file_path,'rb') as f:pass
        process_video_for_court_boundary(video_file_path)
    except FileNotFoundError: print(f"Lỗi: Không tìm thấy file '{video_file_path}'.")
    except Exception as e: print(f"Lỗi không mong muốn: {e}")

100%|██████████| 6.25M/6.25M [00:00<00:00, 10.2MB/s]


Đã nhấn 'q' trong quá trình xử lý video. Thoát.
Xử lý hoàn tất. Tổng frames: 53, Tổng thời gian: 7.34s, FPS trung bình: 7.22
